# ♊ Installing Required Libraries and Configs

In [ ]:
! pip install trl peft accelerate datasets bitsandbytes git+https://github.com/huggingface/transformers

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
from accelerate import Accelerator
import os

# ♊  Preparing The Dataset

In [ ]:
# #save final Dataset to the drive
# df.to_csv('/content/drive/MyDrive/IIT_Criminal_Cases/Data/final_dataset.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

FINAL_DATASET_DIR = '/content/drive/MyDrive/RL_recommendation-zaid/DATA/output.csv'
df = pd.read_csv(FINAL_DATASET_DIR)
df.head()

,prompt,text
0,Suppose there is a user with the following det...,Hey there! I see that you're feeling frustrate...
1,Suppose there is a user with the following det...,"Dear Teacher,\n\nTaking a break and engaging i..."
2,Suppose there is a user with the following det...,Taking a break from work and going on a trip c...
3,Suppose there is a user with the following det...,Learning to cook something new can not only ex...
4,Suppose there is a user with the following det...,"Dear User,\n\nBased on your details, it seems ..."


In [ ]:
df["query"] = df[["prompt", "text"]].apply(lambda x: "Human: " + x["prompt"] + " " + " Assistant: "+ x["text"], axis=1)
df.head()

,prompt,text,query
0,Suppose there is a user with the following det...,Hey there! I see that you're feeling frustrate...,Human: Suppose there is a user with the follow...
1,Suppose there is a user with the following det...,"Dear Teacher,\n\nTaking a break and engaging i...",Human: Suppose there is a user with the follow...
2,Suppose there is a user with the following det...,Taking a break from work and going on a trip c...,Human: Suppose there is a user with the follow...
3,Suppose there is a user with the following det...,Learning to cook something new can not only ex...,Human: Suppose there is a user with the follow...
4,Suppose there is a user with the following det...,"Dear User,\n\nBased on your details, it seems ...",Human: Suppose there is a user with the follow...


In [ ]:
data = Dataset.from_pandas(df)

# ✈ Gemma - 7b-it Training

In [ ]:
sft_config = {
            "model_ckpt": "google/gemma-2b-it",
            "load_in_4bit": True,
            "device_map": {"": Accelerator().local_process_index},
            "torch_dtype": torch.float16,
            "trust_remote_code": True,
            "use_lora": True,
            "r": 16,
            "lora_alpha": 16,
            "lora_dropout": 0.05,
            "bias": "none",
            "task_type": "CAUSAL_LM",
            "target_modules": ["q_proj", "v_proj"],
            "output_dir": "zaid-gemma01",
            "per_device_train_batch_size": 1,
            "gradient_accumulation_steps": 1,
            "optim": "paged_adamw_32bit",
            "learning_rate": 2e-4,
            "lr_scheduler_type": "cosine",
            "save_strategy": "epoch",
            "logging_steps": 50,
            "num_train_epochs": 5,
            "max_steps": 500,
            "fp16": True,
            "push_to_hub": True,
            "train_cln_name": "text",
            "packing": False,
            "max_seq_length": 512,
            "neftune_noise_alpha": 5
        }

In [ ]:
class TrainSFT:

    def __init__(self, data, config):
        self.data = data
        self.config = config
        self.writer = SummaryWriter(log_dir=self.config["output_dir"])

    def prepare_lora_model(self):

        self.lora_config = LoraConfig(
                                    r=self.config["r"],
                                    lora_alpha=self.config["lora_alpha"],
                                    lora_dropout=self.config["lora_dropout"],
                                    bias=self.config["bias"],
                                    task_type=self.config["task_type"],
                                    target_modules=self.config["target_modules"]
                                )
        self.model = get_peft_model(self.model, self.lora_config)
    def load_model_tokenizer(self):

        self.model = AutoModelForCausalLM.from_pretrained(
                            self.config["model_ckpt"],
                            load_in_4bit=self.config["load_in_4bit"],
                            device_map=self.config["device_map"],
                            torch_dtype=self.config["torch_dtype"]
                        )
        self.model.config.use_cache=False
        self.model.config.pretraining_tp=1
        self.model = prepare_model_for_kbit_training(self.model)
        if self.config["use_lora"]:
            self.prepare_lora_model()

        self.tokenizer = AutoTokenizer.from_pretrained(self.config["model_ckpt"])
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def set_training_args(self):

        return TrainingArguments(
                                    output_dir=self.config["output_dir"],
                                    per_device_train_batch_size=self.config["per_device_train_batch_size"],
                                    gradient_accumulation_steps=self.config["gradient_accumulation_steps"],
                                    optim=self.config["optim"],
                                    learning_rate=self.config["learning_rate"],
                                    lr_scheduler_type=self.config["lr_scheduler_type"],
                                    save_strategy=self.config["save_strategy"],
                                    logging_steps=self.config["logging_steps"],
                                    num_train_epochs=self.config["num_train_epochs"],
                                    max_steps=self.config["max_steps"],
                                    fp16=self.config["fp16"],
                                    push_to_hub=self.config["push_to_hub"]
                                )

    def create_trainer(self):

        self.load_model_tokenizer()
        if self.config["use_lora"]:
            print(self.model.print_trainable_parameters())
            self.trainer = SFTTrainer(
                                    model=self.model,
                                    train_dataset=self.data,
                                    peft_config=self.lora_config,
                                    dataset_text_field=self.config["train_cln_name"],
                                    args=self.set_training_args(),
                                    tokenizer=self.tokenizer,
                                    packing=self.config["packing"],
                                    max_seq_length=self.config["max_seq_length"]
                                )
        else:
            self.trainer = SFTTrainer(
                                    model=self.model,
                                    train_dataset=self.data,
                                    dataset_text_field=self.config["train_cln_name"],
                                    args=self.set_training_args(),
                                    tokenizer=self.tokenizer,
                                    packing=self.config["packing"],
                                    max_seq_length=self.config["max_seq_length"]
                                )

    def train_and_save_model(self):
        self.create_trainer()
        self.trainer.train()
        self.trainer.save_model(self.config["output_dir"])
        self.tokenizer.save_pretrained(self.config["output_dir"])

In [ ]:
train_sft = TrainSFT(data, sft_config)
train_sft.train_and_save_model()

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

trainable params: 1,843,200 || all params: 2,508,015,616 || trainable%: 0.073492365368111
None


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Li

Step,Training Loss
50,2.232000
100,1.648800
150,1.244700
200,0.899100
250,0.654800
300,0.425200
350,0.292300
400,0.228700
450,0.202600
500,0.178100


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

# Inference the Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

FINAL_DATASET_DIR = '/content/drive/MyDrive/RL_recommendation-zaid/DATA/output.csv'
df = pd.read_csv(FINAL_DATASET_DIR)
df.head()

,prompt,text
0,Suppose there is a user with the following det...,Hey there! I see that you're feeling frustrate...
1,Suppose there is a user with the following det...,"Dear Teacher,\n\nTaking a break and engaging i..."
2,Suppose there is a user with the following det...,Taking a break from work and going on a trip c...
3,Suppose there is a user with the following det...,Learning to cook something new can not only ex...
4,Suppose there is a user with the following det...,"Dear User,\n\nBased on your details, it seems ..."


In [ ]:
# ! pip install trl peft accelerate datasets bitsandbytes git+https://github.com/huggingface/transformers -q

##

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from transformers import AutoTokenizer
import torch

MODEL_PATH = 'LahiruProjects/zaid-gemma01'

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

model = AutoPeftModelForCausalLM.from_pretrained(
    MODEL_PATH,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="cuda",
    load_in_4bit=True)

generation_config = GenerationConfig(
    max_new_tokens=500,
    pad_token_id=tokenizer.eos_token_id
)

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/7.38M [00:00<?, ?B/s]

In [ ]:
Query = "###Human: "+ df['prompt'][5]+"###Assistant: "
inputs = tokenizer(Query, return_tensors="pt").to("cuda")


outputs = model.generate(**inputs, generation_config=generation_config)
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
assistant_output = decoded_output.split("###Assistant:")[1].strip()
assistant_output

'The suggestion to go spend time in the ground playing sports or engaging in likeable games is a great idea for improving overall well-being. It will help in relieving stress, boosting focus, and promoting relaxation. By engaging in physical activity, you can clear your mind, release any pent-up emotions, and improve your overall mental and emotional state. This suggestion encourages you to find a balance between work and leisure activities to maintain a healthy balance and find joy in activities that bring you happiness.'

In [ ]:
Query = "###Human: "+ df['prompt'][4]+"###Assistant: "
inputs = tokenizer(Query, return_tensors="pt").to("cuda")


outputs = model.generate(**inputs, generation_config=generation_config)
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
assistant_output = decoded_output.split("###Assistant:")[1].strip()
assistant_output

'Based on the user\'s details, it seems like spending time in the gym could be beneficial for you. Regular exercise not only improves physical health but also helps in boosting confidence and reducing feelings of insecurity. By focusing on your fitness goals and consistently working towards them, you can build more self-assurance and feel better about yourself. Additionally, staying active in the gym can also help in overcoming negative emotions and stress, creating a positive impact on your overall well-being. So, why not give it a try and see how it can positively impact your life. Best regards, [Your System]"\n\nThis suggestion can encourage the user to take care of himself both physically and mentally. By dedicating time to exercise, he can improve his confidence and reduce feelings of insecurity. Additionally, staying active in the gym can help in boosting his confidence and reducing feelings of insecurity.'

In [ ]:
Query = "###Human: "+ df['prompt'][14]+"###Assistant: "
inputs = tokenizer(Query, return_tensors="pt").to("cuda")


outputs = model.generate(**inputs, generation_config=generation_config)
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
assistant_output = decoded_output.split("###Assistant:")[1].strip()
assistant_output

'Spending a few hours in meditation and yoga can help alleviate the negative state of emotion caused by recurring bugs and errors in your codes. By practicing these techniques, you may find that you are better able to clear your mind, reduce stress, and improve your focus. This can lead to increased productivity and improved performance in your work.'